# RNA-Seq Analysis Training Demo (Snakemake)

## Overview

This short tutorial demonstrates how to run an RNA-Seq workflow using a prokaryotic data set. Steps in the workflow include read trimming, read QC, read mapping, and counting mapped reads per gene to quantitate gene expression. This tutorial uses a popular workflow manager called 'snakemake'. More information on snakemake can be found <a href="https://snakemake.readthedocs.io/en/stable/">here</a>. Running the code in this tutorial will take approximately 12 minutes.

![RNA-Seq workflow](images/rnaseq-workflow.png)

### STEP 1: Install mambaforge and snakemake
First install mambaforge.

We will use it to install snakemake, as well as create a snakemake environment using mambaforge.

In [1]:
!curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-$(uname)-$(uname -m).sh
!bash Mambaforge-$(uname)-$(uname -m).sh -b -p $HOME/mambaforge
!$HOME/mambaforge/bin/mamba install -y -c conda-forge -c bioconda snakemake
!cp /home/jupyter/mambaforge/bin/mamba /opt/conda/bin

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 88.6M  100 88.6M    0     0  26.0M      0  0:00:03  0:00:03 --:--:-- 30.4M
PREFIX=/home/jupyter/mambaforge
Unpacking payload ...
Extracting "lz4-c-1.9.3-h9c3ff4c_1.tar.bz2"
Extracting "conda-4.12.0-py39hf3d152e_0.tar.bz2"
Extracting "pip-22.0.4-pyhd8ed1ab_0.tar.bz2"
Extracting "tzdata-2022a-h191b570_0.tar.bz2"
Extracting "pyopenssl-22.0.0-pyhd8ed1ab_0.tar.bz2"
Extracting "ruamel_yaml-0.15.80-py39h3811e60_1006.tar.bz2"
Extracting "cffi-1.15.0-py39h4bc2ebd_0.tar.bz2"
Extracting "ca-certificates-2021.10.8-ha878542_0.tar.bz2"
Extracting "pybind11-abi-4-hd8ed1ab_3.tar.bz2"
Extracting "libxml2-2.9.12-h885dcf4_1.tar.bz2"
Extracting "six-1.16.0-pyh6c4a22f_0.tar.bz2"
Extracting "l

### STEP 2: Create directories that will be used in our analysis

In [2]:
!echo $PWD
!mkdir -p data
!mkdir -p data/raw_fastq
!mkdir -p data/trimmed
!mkdir -p data/reference
!mkdir -p data/fastqc
!mkdir -p envs

/home/jupyter/rnaseq-myco-notebook


### STEP 3: Copy FASTQ Files
In order for this tutorial to run quickly, we will only analyze 50,000 reads from a sample from both sample groupsinstead of analyzing all the reads from all six samples. These files have been posted on a Google Storage Bucket that we made publicly accessible.


In [3]:
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/data/raw_fastqSub/SRR13349122_1.fastq --output data/raw_fastq/SRR13349122_1.fastq
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/data/raw_fastqSub/SRR13349122_2.fastq --output data/raw_fastq/SRR13349122_2.fastq
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/data/raw_fastqSub/SRR13349128_1.fastq --output data/raw_fastq/SRR13349128_1.fastq
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/data/raw_fastqSub/SRR13349128_2.fastq --output data/raw_fastq/SRR13349128_2.fastq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 8452k  100 8452k    0     0  30.4M      0 --:--:-- --:--:-- --:--:-- 30.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 8452k  100 8452k    0     0  31.6M      0 --:--:-- --:--:-- --:--:-- 31.6M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 8452k  100 8452k    0     0  32.6M      0 --:--:-- --:--:-- --:--:-- 32.6M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 8452k  100 8452k    0     0  12.3M      0 --:--:-- --:--:-- --:--:-- 12.3M


### STEP 4: Copy reference transcriptome files that will be used by Salmon
Salmon is a tool that aligns RNA-Seq reads to a set of transcripts rather than the entire genome.

In [4]:
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/data/reference/M_chelonae_transcripts.fasta --output data/reference/M_chelonae_transcripts.fasta
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/data/reference/decoys.txt --output data/reference/decoys.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9599k  100 9599k    0     0  9785k      0 --:--:-- --:--:-- --:--:-- 9775k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    14  100    14    0     0     81      0 --:--:-- --:--:-- --:--:--    81


### STEP 5: Copy data file for Trimmomatic

In [5]:
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/config/TruSeq3-PE.fa --output TruSeq3-PE.fa

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    95  100    95    0     0    922      0 --:--:-- --:--:-- --:--:--   922


### STEP 6: Download data and config files that will be used in our snakemake environment

Next download config files for our snakemake environment, as well as data files which we will analyze.

In [6]:
# Copy config and data files

!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/config.yaml --output config.yaml
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/snakefile --output snakefile
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/envs/fastqc.yaml --output envs/fastqc.yaml
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/envs/trimmomatic.yaml --output envs/trimmomatic.yaml
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/envs/salmon.yaml --output envs/salmon.yaml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    67  100    67    0     0    426      0 --:--:-- --:--:-- --:--:--   426
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3561  100  3561    0     0  19353      0 --:--:-- --:--:-- --:--:-- 19353
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   106  100   106    0     0    642      0 --:--:-- --:--:-- --:--:--   642
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   106  100   106    0     0    585      0 --:--:-- --:--:-- --:--:--   585
  % Total    % Received % Xferd  Average Speed   Tim

#### Explanation of config files

Below is a brief explanation of the config files:

In [7]:
!printf "The config.yaml file contains our sample names:\n\n Config.yaml\n"
!cat config.yaml
!printf "\n\nThe env folder contains information pertaining to packages to be used in the environment, \nas well as their version, for example, here is the 'envs/fastqc.yaml' file:\n\n Fastqc.Yaml\n"
!cat envs/fastqc.yaml

The config.yaml file contains our sample names:

 Config.yaml
samples:
    SRR13349122: SRR13349122
    SRR13349128: SRR13349128


The env folder contains information pertaining to packages to be used in the environment, 
as well as their version, for example, here is the 'envs/fastqc.yaml' file:

 Fastqc.Yaml
channels:
  - bioconda
  - conda-forge
  - defaults
dependencies:
  - fastqc ==0.11.9
  - multiqc ==1.12



### STEP 7: Run snakemake on our snakefile

In [41]:
!snakemake --cores --use-conda --forceall

{'SRR13349122': 'SRR13349122', 'SRR13349128': 'SRR13349128'}
Building DAG of jobs...
Creating conda environment envs/trimmomatic.yaml...
Environment for /home/jupyter/rnaseq-myco-notebook/envs/trimmomatic.yaml created (location: .snakemake/conda/c5c104d5cc0c87027ce7881076c6b53e)
Creating conda environment envs/fastqc.yaml...
Environment for /home/jupyter/rnaseq-myco-notebook/envs/fastqc.yaml created (location: .snakemake/conda/7c2a231706a71b566b29f65dd3d644bf)
Creating conda environment envs/salmon.yaml...
Environment for /home/jupyter/rnaseq-myco-notebook/envs/salmon.yaml created (location: .snakemake/conda/d4f33c5881f8b4f90819e2e3764f2eb5)
Using shell: /usr/bin/bash
Provided cores: 4
Rules claiming more threads will be scaled down.
Job stats:
job                   count    min threads    max threads
------------------  -------  -------------  -------------
all                       1              1              1
fastqc_trimmed            1              4              4
multiqc_trimm

## <a name="workflow">Additional Workflows</a>

Now that you have read counts per gene, feel free to explore the R workflow which creates plots and analyses using these readcount files, or try other alternate workflows for creating read count files, such as the standard short or extended tutorials.


[Workflow One:](Tutorial_1.ipynb) A short introduction to downloading and mapping sequences to a transcriptome using Trimmomatic and Salmon. Here is a link to the YouTube video demonstrating the tutorial: <https://www.youtube.com/watch?v=NG1U7D4l31o&t=26s>.

[Workflow One (Extended):](Tutorial_1B_Extended.ipynb) An extended version of workflow one. Once you have got your feet wet, you can retry workflow one with this extended version that covers the entire dataset, and includes elaboration such as using SRA tools for sequence downloading, and examples of running batches of fastq files through the pipeline. This workflow may take around an hour to run.

[Workflow One (Using Snakemake):](Tutorial_2_Snakemake.ipynb) Using snakemake to run workflow one.

[Workflow Two (DEG Analysis):](Tutorial_3_DEG_Analysis.ipynb) Using Deseq2 and R to conduct clustering and differential gene expression analysis.


![RNA-Seq workflow](images/RNA-Seq_Notebook_Homepage.png)